In [347]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing as prep
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
import re
from datetime import datetime

In [348]:
# Brining in data file
df = pd.read_csv('Uber Request Data.csv')

In [349]:
# Verifying import
df.head()

,Request id,Pickup point,Driver id,Status,Request timestamp,Drop timestamp
0,619,Airport,1.0,Trip Completed,11/7/2016 11:51,11/7/2016 13:00
1,867,Airport,1.0,Trip Completed,11/7/2016 17:57,11/7/2016 18:47
2,1807,City,1.0,Trip Completed,12/7/2016 9:17,12/7/2016 9:58
3,2532,Airport,1.0,Trip Completed,12/7/2016 21:08,12/7/2016 22:03
4,3112,City,1.0,Trip Completed,13-07-2016 08:33:16,13-07-2016 09:25:47


In [350]:
# Finding all possible outcomes for 'Status' column
df.Status.unique()

array(['Trip Completed', 'Cancelled', 'No Cars Available'], dtype=object)

In [351]:
# Replacing spaces in columns with underscores
df.columns = df.columns.str.replace(' ','_')
df.head()

,Request_id,Pickup_point,Driver_id,Status,Request_timestamp,Drop_timestamp
0,619,Airport,1.0,Trip Completed,11/7/2016 11:51,11/7/2016 13:00
1,867,Airport,1.0,Trip Completed,11/7/2016 17:57,11/7/2016 18:47
2,1807,City,1.0,Trip Completed,12/7/2016 9:17,12/7/2016 9:58
3,2532,Airport,1.0,Trip Completed,12/7/2016 21:08,12/7/2016 22:03
4,3112,City,1.0,Trip Completed,13-07-2016 08:33:16,13-07-2016 09:25:47


In [352]:
# Splitting Request_timestamp into individual columns
df[['Req_Date','Request_Time']] = df.Request_timestamp.str.split(" ",expand=True)
df.head()


,Request_id,Pickup_point,Driver_id,Status,Request_timestamp,Drop_timestamp,Req_Date,Request_Time
0,619,Airport,1.0,Trip Completed,11/7/2016 11:51,11/7/2016 13:00,11/7/2016,11:51
1,867,Airport,1.0,Trip Completed,11/7/2016 17:57,11/7/2016 18:47,11/7/2016,17:57
2,1807,City,1.0,Trip Completed,12/7/2016 9:17,12/7/2016 9:58,12/7/2016,9:17
3,2532,Airport,1.0,Trip Completed,12/7/2016 21:08,12/7/2016 22:03,12/7/2016,21:08
4,3112,City,1.0,Trip Completed,13-07-2016 08:33:16,13-07-2016 09:25:47,13-07-2016,08:33:16


In [353]:
# Splitting Drop_timestamp into individual columns
df[['Drop_Date','Drop_Time']] = df.Drop_timestamp.str.split(" ",expand=True)
df.head()

,Request_id,Pickup_point,Driver_id,Status,Request_timestamp,Drop_timestamp,Req_Date,Request_Time,Drop_Date,Drop_Time
0,619,Airport,1.0,Trip Completed,11/7/2016 11:51,11/7/2016 13:00,11/7/2016,11:51,11/7/2016,13:00
1,867,Airport,1.0,Trip Completed,11/7/2016 17:57,11/7/2016 18:47,11/7/2016,17:57,11/7/2016,18:47
2,1807,City,1.0,Trip Completed,12/7/2016 9:17,12/7/2016 9:58,12/7/2016,9:17,12/7/2016,9:58
3,2532,Airport,1.0,Trip Completed,12/7/2016 21:08,12/7/2016 22:03,12/7/2016,21:08,12/7/2016,22:03
4,3112,City,1.0,Trip Completed,13-07-2016 08:33:16,13-07-2016 09:25:47,13-07-2016,08:33:16,13-07-2016,09:25:47


In [354]:
# Dropping redundancy
df = df.drop(columns=['Request_timestamp', 'Drop_timestamp'])


In [355]:
# Converting status options to a mapped value, 1 = completed 2 = incomplete
outcomes = {
    'Trip Completed' : 1, 
    'No Cars Available' : 0, 
    'Cancelled' : 0, 
}

# Adding new column for mapped options
df['mapped_status_type'] = df['Status'].map(outcomes)

# Testing new column placement
df.head()

,Request_id,Pickup_point,Driver_id,Status,Req_Date,Request_Time,Drop_Date,Drop_Time,mapped_status_type
0,619,Airport,1.0,Trip Completed,11/7/2016,11:51,11/7/2016,13:00,1
1,867,Airport,1.0,Trip Completed,11/7/2016,17:57,11/7/2016,18:47,1
2,1807,City,1.0,Trip Completed,12/7/2016,9:17,12/7/2016,9:58,1
3,2532,Airport,1.0,Trip Completed,12/7/2016,21:08,12/7/2016,22:03,1
4,3112,City,1.0,Trip Completed,13-07-2016,08:33:16,13-07-2016,09:25:47,1


In [356]:
df.isnull().sum()

Request_id               0
Pickup_point             0
Driver_id             2650
Status                   0
Req_Date                 0
Request_Time             0
Drop_Date             3914
Drop_Time             3914
mapped_status_type       0
dtype: int64

### We will not be removing null values as those are crucial to the determination of "Status" in our model

In [357]:
df.Req_Date.apply(lambda x: pd.to_datetime(x).strftime('%m/%d/%y')[0])
df.head()

,Request_id,Pickup_point,Driver_id,Status,Req_Date,Request_Time,Drop_Date,Drop_Time,mapped_status_type
0,619,Airport,1.0,Trip Completed,11/7/2016,11:51,11/7/2016,13:00,1
1,867,Airport,1.0,Trip Completed,11/7/2016,17:57,11/7/2016,18:47,1
2,1807,City,1.0,Trip Completed,12/7/2016,9:17,12/7/2016,9:58,1
3,2532,Airport,1.0,Trip Completed,12/7/2016,21:08,12/7/2016,22:03,1
4,3112,City,1.0,Trip Completed,13-07-2016,08:33:16,13-07-2016,09:25:47,1


In [358]:
df.Req_Date.unique()

array(['11/7/2016', '12/7/2016', '13-07-2016', '14-07-2016', '15-07-2016'],
      dtype=object)

In [359]:
# Replacing hyphens with '/' in appropriate columns
pd.Series(['Req_Date', 'Drop_Date', np.nan]).str.replace('-', '/', regex=True)
df.head()

,Request_id,Pickup_point,Driver_id,Status,Req_Date,Request_Time,Drop_Date,Drop_Time,mapped_status_type
0,619,Airport,1.0,Trip Completed,11/7/2016,11:51,11/7/2016,13:00,1
1,867,Airport,1.0,Trip Completed,11/7/2016,17:57,11/7/2016,18:47,1
2,1807,City,1.0,Trip Completed,12/7/2016,9:17,12/7/2016,9:58,1
3,2532,Airport,1.0,Trip Completed,12/7/2016,21:08,12/7/2016,22:03,1
4,3112,City,1.0,Trip Completed,13-07-2016,08:33:16,13-07-2016,09:25:47,1


In [360]:
df.Req_Date.unique()

array(['11/7/2016', '12/7/2016', '13-07-2016', '14-07-2016', '15-07-2016'],
      dtype=object)

In [361]:
df.Req_Date.replace("-","/")
df.head()

,Request_id,Pickup_point,Driver_id,Status,Req_Date,Request_Time,Drop_Date,Drop_Time,mapped_status_type
0,619,Airport,1.0,Trip Completed,11/7/2016,11:51,11/7/2016,13:00,1
1,867,Airport,1.0,Trip Completed,11/7/2016,17:57,11/7/2016,18:47,1
2,1807,City,1.0,Trip Completed,12/7/2016,9:17,12/7/2016,9:58,1
3,2532,Airport,1.0,Trip Completed,12/7/2016,21:08,12/7/2016,22:03,1
4,3112,City,1.0,Trip Completed,13-07-2016,08:33:16,13-07-2016,09:25:47,1


In [362]:
# Converting all request date formats to be the same
df['Req_Date'] = pd.to_datetime(df.Req_Date)
df['Req_Date'] = df['Req_Date'].dt.strftime('%m/%d/%Y')
df.head()

,Request_id,Pickup_point,Driver_id,Status,Req_Date,Request_Time,Drop_Date,Drop_Time,mapped_status_type
0,619,Airport,1.0,Trip Completed,11/07/2016,11:51,11/7/2016,13:00,1
1,867,Airport,1.0,Trip Completed,11/07/2016,17:57,11/7/2016,18:47,1
2,1807,City,1.0,Trip Completed,12/07/2016,9:17,12/7/2016,9:58,1
3,2532,Airport,1.0,Trip Completed,12/07/2016,21:08,12/7/2016,22:03,1
4,3112,City,1.0,Trip Completed,07/13/2016,08:33:16,13-07-2016,09:25:47,1


In [363]:
# Converting all drop date formats to be the same
df['Drop_Date'] = pd.to_datetime(df.Req_Date)
df['Drop_Date'] = df['Drop_Date'].dt.strftime('%m/%d/%Y')
df.head()

,Request_id,Pickup_point,Driver_id,Status,Req_Date,Request_Time,Drop_Date,Drop_Time,mapped_status_type
0,619,Airport,1.0,Trip Completed,11/07/2016,11:51,11/07/2016,13:00,1
1,867,Airport,1.0,Trip Completed,11/07/2016,17:57,11/07/2016,18:47,1
2,1807,City,1.0,Trip Completed,12/07/2016,9:17,12/07/2016,9:58,1
3,2532,Airport,1.0,Trip Completed,12/07/2016,21:08,12/07/2016,22:03,1
4,3112,City,1.0,Trip Completed,07/13/2016,08:33:16,07/13/2016,09:25:47,1


In [364]:
# Converting request time to string
Request_Time = df.Request_Time.astype(str).str.zfill(4)
Drop_Time = df.Drop_Time.astype(str).str.zfill(4)

In [365]:
df.head()

,Request_id,Pickup_point,Driver_id,Status,Req_Date,Request_Time,Drop_Date,Drop_Time,mapped_status_type
0,619,Airport,1.0,Trip Completed,11/07/2016,11:51,11/07/2016,13:00,1
1,867,Airport,1.0,Trip Completed,11/07/2016,17:57,11/07/2016,18:47,1
2,1807,City,1.0,Trip Completed,12/07/2016,9:17,12/07/2016,9:58,1
3,2532,Airport,1.0,Trip Completed,12/07/2016,21:08,12/07/2016,22:03,1
4,3112,City,1.0,Trip Completed,07/13/2016,08:33:16,07/13/2016,09:25:47,1


In [366]:
# Splitting time into three columns so we can convert all entries to same format
df[['ReqHour','ReqMinute','RegSecond']] = df['Request_Time'].str.split(':',expand=True)


In [367]:
df.head()

,Request_id,Pickup_point,Driver_id,Status,Req_Date,Request_Time,Drop_Date,Drop_Time,mapped_status_type,ReqHour,ReqMinute,RegSecond
0,619,Airport,1.0,Trip Completed,11/07/2016,11:51,11/07/2016,13:00,1,11,51,None
1,867,Airport,1.0,Trip Completed,11/07/2016,17:57,11/07/2016,18:47,1,17,57,None
2,1807,City,1.0,Trip Completed,12/07/2016,9:17,12/07/2016,9:58,1,9,17,None
3,2532,Airport,1.0,Trip Completed,12/07/2016,21:08,12/07/2016,22:03,1,21,08,None
4,3112,City,1.0,Trip Completed,07/13/2016,08:33:16,07/13/2016,09:25:47,1,08,33,16


In [368]:
# Aggregating Hour and Minute as seconds is not neccesary to our project
df['Req_Time'] = df[['ReqHour','ReqMinute']].agg(':'.join, axis=1)


df.head()

,Request_id,Pickup_point,Driver_id,Status,Req_Date,Request_Time,Drop_Date,Drop_Time,mapped_status_type,ReqHour,ReqMinute,RegSecond,Req_Time
0,619,Airport,1.0,Trip Completed,11/07/2016,11:51,11/07/2016,13:00,1,11,51,None,11:51
1,867,Airport,1.0,Trip Completed,11/07/2016,17:57,11/07/2016,18:47,1,17,57,None,17:57
2,1807,City,1.0,Trip Completed,12/07/2016,9:17,12/07/2016,9:58,1,9,17,None,9:17
3,2532,Airport,1.0,Trip Completed,12/07/2016,21:08,12/07/2016,22:03,1,21,08,None,21:08
4,3112,City,1.0,Trip Completed,07/13/2016,08:33:16,07/13/2016,09:25:47,1,08,33,16,08:33


In [369]:
# Dropping old request time column
df = df.drop('Request_Time', 1)


/var/folders/6t/wkk2qcbx02q62hf6jd90h07m0000gn/T/ipykernel_81200/2798230673.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('Request_Time', 1)


In [370]:
# Dropping split hour column
df = df.drop('ReqHour', 1)

/var/folders/6t/wkk2qcbx02q62hf6jd90h07m0000gn/T/ipykernel_81200/356063247.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('ReqHour', 1)


In [371]:
# Dropping split minute column
df = df.drop('ReqMinute', 1)

/var/folders/6t/wkk2qcbx02q62hf6jd90h07m0000gn/T/ipykernel_81200/3292605984.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('ReqMinute', 1)


In [372]:
# Dropping split second column
df = df.drop('RegSecond', 1)

/var/folders/6t/wkk2qcbx02q62hf6jd90h07m0000gn/T/ipykernel_81200/3051739366.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('RegSecond', 1)


In [373]:
# Verifying df format is accurate
df.head()

,Request_id,Pickup_point,Driver_id,Status,Req_Date,Drop_Date,Drop_Time,mapped_status_type,Req_Time
0,619,Airport,1.0,Trip Completed,11/07/2016,11/07/2016,13:00,1,11:51
1,867,Airport,1.0,Trip Completed,11/07/2016,11/07/2016,18:47,1,17:57
2,1807,City,1.0,Trip Completed,12/07/2016,12/07/2016,9:58,1,9:17
3,2532,Airport,1.0,Trip Completed,12/07/2016,12/07/2016,22:03,1,21:08
4,3112,City,1.0,Trip Completed,07/13/2016,07/13/2016,09:25:47,1,08:33


In [374]:
# Re-mapping column locations for organization
df = df[['Request_id','Pickup_point','Status','mapped_status_type','Req_Date','Req_Time','Drop_Date','Drop_Time']]

In [375]:

df.head()

,Request_id,Pickup_point,Status,mapped_status_type,Req_Date,Req_Time,Drop_Date,Drop_Time
0,619,Airport,Trip Completed,1,11/07/2016,11:51,11/07/2016,13:00
1,867,Airport,Trip Completed,1,11/07/2016,17:57,11/07/2016,18:47
2,1807,City,Trip Completed,1,12/07/2016,9:17,12/07/2016,9:58
3,2532,Airport,Trip Completed,1,12/07/2016,21:08,12/07/2016,22:03
4,3112,City,Trip Completed,1,07/13/2016,08:33,07/13/2016,09:25:47


In [376]:
import datetime
def is_weekday(dt):
    dow = datetime.datetime.strptime(dt, '%m/%d/%Y').isoweekday()
    if dow == 6 or dow == 7:
        return 0
    else:
        return 1

In [377]:
df['is_weekday'] = df['Req_Date'].apply(lambda x: is_weekday(x))
df.head()

,Request_id,Pickup_point,Status,mapped_status_type,Req_Date,Req_Time,Drop_Date,Drop_Time,is_weekday
0,619,Airport,Trip Completed,1,11/07/2016,11:51,11/07/2016,13:00,1
1,867,Airport,Trip Completed,1,11/07/2016,17:57,11/07/2016,18:47,1
2,1807,City,Trip Completed,1,12/07/2016,9:17,12/07/2016,9:58,1
3,2532,Airport,Trip Completed,1,12/07/2016,21:08,12/07/2016,22:03,1
4,3112,City,Trip Completed,1,07/13/2016,08:33,07/13/2016,09:25:47,1


In [379]:
df.head()

,Request_id,Pickup_point,Status,mapped_status_type,Req_Date,Req_Time,Drop_Date,Drop_Time,is_weekday
0,619,Airport,Trip Completed,1,11/07/2016,11:51,11/07/2016,13:00,1
1,867,Airport,Trip Completed,1,11/07/2016,17:57,11/07/2016,18:47,1
2,1807,City,Trip Completed,1,12/07/2016,9:17,12/07/2016,9:58,1
3,2532,Airport,Trip Completed,1,12/07/2016,21:08,12/07/2016,22:03,1
4,3112,City,Trip Completed,1,07/13/2016,08:33,07/13/2016,09:25:47,1


In [380]:
df[['DropHour','DropMinute','DropSeconds']] = df['Drop_Time'].str.split(':',expand=True)


In [381]:
df.head()

,Request_id,Pickup_point,Status,mapped_status_type,Req_Date,Req_Time,Drop_Date,Drop_Time,is_weekday,DropHour,DropMinute,DropSeconds
0,619,Airport,Trip Completed,1,11/07/2016,11:51,11/07/2016,13:00,1,13,00,None
1,867,Airport,Trip Completed,1,11/07/2016,17:57,11/07/2016,18:47,1,18,47,None
2,1807,City,Trip Completed,1,12/07/2016,9:17,12/07/2016,9:58,1,9,58,None
3,2532,Airport,Trip Completed,1,12/07/2016,21:08,12/07/2016,22:03,1,22,03,None
4,3112,City,Trip Completed,1,07/13/2016,08:33,07/13/2016,09:25:47,1,09,25,47


In [382]:
df = df.drop('Drop_Time', 1)


/var/folders/6t/wkk2qcbx02q62hf6jd90h07m0000gn/T/ipykernel_81200/2322730204.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('Drop_Time', 1)


In [383]:
df['Drop_Time'] = df[['DropHour','DropMinute']].agg(':'.join, axis=1)
df.head()

TypeError: sequence item 0: expected str instance, float found

In [393]:
df['Drop_Time'] = [':'.join(str(x) for x in y) for y in map(tuple, df[['DropHour', 'DropMinute']].values)]
df.head()

,Request_id,Pickup_point,Status,mapped_status_type,Req_Date,Req_Time,Drop_Date,is_weekday,DropHour,DropMinute,DropSeconds,Drop_Time
0,619,Airport,Trip Completed,1,11/07/2016,11:51,11/07/2016,1,13,00,None,13:00
1,867,Airport,Trip Completed,1,11/07/2016,17:57,11/07/2016,1,18,47,None,18:47
2,1807,City,Trip Completed,1,12/07/2016,9:17,12/07/2016,1,9,58,None,9:58
3,2532,Airport,Trip Completed,1,12/07/2016,21:08,12/07/2016,1,22,03,None,22:03
4,3112,City,Trip Completed,1,07/13/2016,08:33,07/13/2016,1,09,25,47,09:25


In [394]:
df = df.drop('DropHour', 1)
df = df.drop('DropMinute', 1)
df = df.drop('DropSeconds', 1)

/var/folders/6t/wkk2qcbx02q62hf6jd90h07m0000gn/T/ipykernel_81200/2794547973.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('DropHour', 1)
/var/folders/6t/wkk2qcbx02q62hf6jd90h07m0000gn/T/ipykernel_81200/2794547973.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('DropMinute', 1)
/var/folders/6t/wkk2qcbx02q62hf6jd90h07m0000gn/T/ipykernel_81200/2794547973.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('DropSeconds', 1)


In [398]:
# Re-mapping column to organize again
df = df[['Pickup_point','Status','mapped_status_type','Req_Date','Req_Time','Drop_Date','Drop_Time']]
df.head()

,Pickup_point,Status,mapped_status_type,Req_Date,Req_Time,Drop_Date,Drop_Time
0,Airport,Trip Completed,1,11/07/2016,11:51,11/07/2016,13:00
1,Airport,Trip Completed,1,11/07/2016,17:57,11/07/2016,18:47
2,City,Trip Completed,1,12/07/2016,9:17,12/07/2016,9:58
3,Airport,Trip Completed,1,12/07/2016,21:08,12/07/2016,22:03
4,City,Trip Completed,1,07/13/2016,08:33,07/13/2016,09:25
